# Aprendizaje por refuerzo (posponer al viernes)

![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/refuerzo.png)
Fuente de la imagen: medium.com

“maximizar la recompensa“

**Definiciones importantes:**

- Agente: modelo a entrenar para que tome decisiones.
- Ambiente: Entorno donde interactúa el agente. El ambiente limita y pone reglas. 

Entre ambiente y agente hay las siguientes relaciones: 

- Acción: posibles acciones que puede tomar el agente en determinado momento.
- Estado (del ambiente): indicadores del ambiente sobre cómo están los diversos elementos que lo componen en determinado momento.
- Recompensas (o penalización): cada acción del agente obtiene un premio o una penalización. 

Ver: https://towardsdatascience.com/hands-on-introduction-to-reinforcement-learning-in-python-da07f7aaca88 